<a href="https://colab.research.google.com/github/sgarg99/FacialExpressionClassification/blob/master/ModelEvaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive

In [29]:
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
!pip install -q tf-nightly

In [30]:
cd gdrive/My Drive/Emotion_Classification/PrivateTest

[Errno 2] No such file or directory: 'gdrive/My Drive/Emotion_Classification/PrivateTest'
/content/gdrive/My Drive/Emotion_Classification/PrivateTest


In [0]:
from __future__ import absolute_import, division, print_function
import tensorflow as tf
tf.enable_eager_execution()

tf.VERSION
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [0]:
import pathlib
data_root = "."
data_root = pathlib.Path(data_root)
import random
all_image_paths = list(data_root.glob('*/*'))
all_image_paths = [str(path) for path in all_image_paths]
random.shuffle(all_image_paths)

image_count = len(all_image_paths)
label_names = sorted(item.name for item in data_root.glob('*/') if item.is_dir())
label_to_index = dict((name, index) for index,name in enumerate(label_names))
all_image_labels = [label_to_index[pathlib.Path(path).parent.name]
                    for path in all_image_paths]

def preprocess_image(image):
  image = tf.image.decode_jpeg(image, channels=1)
  image = tf.image.resize_images(image, [48,48])
  image = tf.cast(image , tf.float32)
  image = image/255.0  # normalize to [0,1] range
  return image
def load_and_preprocess_image(path):
  image = tf.read_file(path)
  return preprocess_image(image)

In [78]:
path_ds = tf.data.Dataset.from_tensor_slices(all_image_paths)
image_ds = path_ds.map(load_and_preprocess_image, num_parallel_calls=AUTOTUNE)
label_ds = tf.data.Dataset.from_tensor_slices(tf.cast(all_image_labels, tf.int64))
image_label_ds = tf.data.Dataset.zip((image_ds, label_ds))
BATCH_SIZE = 32

# Setting a shuffle buffer size as large as the dataset ensures that the data is
# completely shuffled.
ds = image_label_ds.shuffle(buffer_size=image_count)

ds = ds.batch(BATCH_SIZE)
# `prefetch` lets the dataset fetch batches, in the background while the model is training.
ds = ds.prefetch(buffer_size=AUTOTUNE)
ds

<DatasetV1Adapter shapes: ((?, 48, 48, 1), (?,)), types: (tf.float32, tf.int64)>

In [0]:
inp_shp=(48,48,1)
model=tf.keras.Sequential()
model.add(tf.keras.layers.Conv2D(32,(3,3),activation="relu",input_shape=inp_shp))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.MaxPooling2D((2,2)))
model.add(tf.keras.layers.Conv2D(48,(3,3),activation="relu"))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.MaxPooling2D((2,2)))
model.add(tf.keras.layers.Conv2D(64,(3,3),activation="relu"))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.MaxPooling2D((2,2)))
model.add(tf.keras.layers.Conv2D(128,(3,3),activation="relu"))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.MaxPooling2D((2,2)))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(256,activation="relu"))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(7,activation="softmax"))

In [0]:
model.load_weights('my_model4.h5')

In [0]:

model.compile(optimizer=tf.keras.optimizers.Adagrad(lr=0.1, epsilon=None, decay=0.0), 
              loss=tf.keras.losses.sparse_categorical_crossentropy,
              metrics=["accuracy"])


In [79]:
model.evaluate(ds)

113/113 [==============================] - 491s 4s/step - loss: 1.3249 - acc: 0.4993


[1.3248807613828542, 0.49930343]

In [73]:
cd ..

/content/gdrive/My Drive/Emotion_Classification


In [67]:
cd Training

/content/gdrive/My Drive/Emotion_Classification/Training


In [0]:
model.load_weights("my_model5.h5")

In [75]:
cd PublicTest

/content/gdrive/My Drive/Emotion_Classification/PublicTest
